## LSTM model

In [1]:
import tensorflow as tf 
import os
import pandas as pd 
import numpy as np 


In [2]:
df=pd.read_csv('BTC_new.csv')

In [3]:
df_as_np_min = None
df_as_np_max = None
def df_to_X_y(df, window_size=5):
    df_as_np=df.to_numpy(dtype=np.float32)
    # get the first column of the numpy array
    df_as_np_min=np.min(df_as_np, axis=0, keepdims=True)
    df_as_np_max = np.max(df_as_np, axis=0, keepdims=True)
    df_as_np_norm=(df_as_np-df_as_np_min)/(df_as_np_max-df_as_np_min)
    y_train=df_as_np_norm[:,0]
    # get the rest of the columns except the first one
    x_train=df_as_np_norm[:,1:5]
    X = []
    y = []
    for i in range(len(df_as_np) - window_size):
        X.append(x_train[i:i+window_size])
        y.append(y_train[i+window_size])
    return np.array(X), np.array(y)
WINDOW_SIZE=5
X,y=df_to_X_y(df, WINDOW_SIZE)
X.shape, y.shape

((3672, 5, 4), (3672,))

In [4]:
WINDOW_SIZE=5
X,y=df_to_X_y(df, WINDOW_SIZE)
X.shape, y.shape
# Normlaize the data


((3672, 5, 4), (3672,))

In [5]:
X_train,y_train=X[:int(0.8*len(X))],y[:int(0.8*len(X))]
X_val, y_val=X[int(0.8*len(X)):int(.9*len(X))],y[int(0.8*len(X)):int(.9*len(X))]
X_test,y_test=X[int(0.9*len(X)):],y[int(0.9*len(X)):]

In [6]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Input
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam

In [7]:
model1=Sequential()
model1.add(LSTM(units=50, return_sequences=True, input_shape=(WINDOW_SIZE,4)))
model1.add(LSTM(units=50, return_sequences=True))
model1.add(Dense(8,"relu"))
model1.add(Dense(1,'linear'))

model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 5, 50)             11000     
                                                                 
 lstm_1 (LSTM)               (None, 5, 50)             20200     
                                                                 
 dense (Dense)               (None, 5, 8)              408       
                                                                 
 dense_1 (Dense)             (None, 5, 1)              9         
                                                                 
Total params: 31,617
Trainable params: 31,617
Non-trainable params: 0
_________________________________________________________________


In [8]:
cp=ModelCheckpoint('model1.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')
model1.compile(loss='mse', optimizer=Adam(lr=0.001), metrics=['mse'])

C:\Users\maniz\anaconda3\envs\py3.7\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [9]:
model1.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100,callbacks=[cp])

Epoch 1/100
87/92 [===========================>..] - ETA: 0s - loss: 0.0577 - mse: 0.0577
Epoch 1: val_loss improved from inf to 0.02151, saving model to model1.h5
92/92 [==============================] - 3s 10ms/step - loss: 0.0573 - mse: 0.0573 - val_loss: 0.0215 - val_mse: 0.0215
Epoch 2/100
81/92 [=========================>....] - ETA: 0s - loss: 0.0539 - mse: 0.0539
Epoch 2: val_loss did not improve from 0.02151
92/92 [==============================] - 0s 4ms/step - loss: 0.0547 - mse: 0.0547 - val_loss: 0.0260 - val_mse: 0.0260
Epoch 3/100
79/92 [========================>.....] - ETA: 0s - loss: 0.0559 - mse: 0.0559
Epoch 3: val_loss improved from 0.02151 to 0.01994, saving model to model1.h5
92/92 [==============================] - 0s 4ms/step - loss: 0.0548 - mse: 0.0548 - val_loss: 0.0199 - val_mse: 0.0199
Epoch 4/100
77/92 [========================>.....] - ETA: 0s - loss: 0.0560 - mse: 0.0560
Epoch 4: val_loss improved from 0.01994 to 0.01537, saving model to model1.h5
92/92

In [16]:
from tensorflow.keras.models import load_model
model1=load_model('model1.h5')

In [17]:
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 5, 50)             11000     
                                                                 
 lstm_1 (LSTM)               (None, 5, 50)             20200     
                                                                 
 dense (Dense)               (None, 5, 8)              408       
                                                                 
 dense_1 (Dense)             (None, 5, 1)              9         
                                                                 
Total params: 31,617
Trainable params: 31,617
Non-trainable params: 0
_________________________________________________________________


In [20]:
#get the predictions
train_predictions=model1.predict(X_train).flatten()
val_predictions=model1.predict(X_val).flatten()
test_predictions=model1.predict(X_test).flatten()

train_result=pd.DataFrame(data={"Train Prediction":train_predictions, "Actuals":y_train})
train_result

# train_predictions=model1.predict(X_train).flatten()
# train_results=pd.DataFrame(data={"Train Predictions":train_predictions, "Train True":y_train})
# train_results